In [ ]:
import pandas as pd

In [ ]:
#Loading data from the Github repository to colab notebook
filename = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter15/Dataset/crx.data'



In [ ]:
# Loading the data using pandas

credData = pd.read_csv(filename,sep=",",header = None,na_values = "?")
credData.head()

In [ ]:
# Changing the Classes to 1 & 0
credData.loc[credData[15] == '+' , 15] = 1
credData.loc[credData[15] == '-' , 15] = 0
credData.head()

In [ ]:
# Dropping all the rows with na values
newcred = credData.dropna(axis = 0)
newcred.shape

In [ ]:
# Seperating X and y variables

X = newcred.loc[:,0:14]
X.shape

In [ ]:
y = newcred.loc[:,15]
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

**Creating processing Engine**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

In [ ]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

**Grid Search**

In [ ]:
# Importing necessary libraries
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# Creating a pipeline with AdaBoostClassifier
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('dimred', PCA()),
                           ('classifier',AdaBoostClassifier(random_state=123))])



In [ ]:
# Defining the parameters as a dictionary
param_grid = {'dimred__n_components':[10,12,15],"classifier__n_estimators": [50, 100,200],"classifier__learning_rate":[0.7,0.6,1.0]}

In [ ]:
from sklearn.model_selection import GridSearchCV
# Fitting the grid search
estimator = GridSearchCV(pipe, cv=10, param_grid=param_grid)


In [ ]:
# Fitting the estimator on the training set
estimator.fit(X_train,y_train)


In [ ]:
# Printing the best score and best parameters
print("Best: %f using %s" % (estimator.best_score_, 
    estimator.best_params_))

In [ ]:
# Predicting with the best estimator
pred = estimator.predict(X_test)

In [ ]:
# Printing the classification report
from sklearn.metrics import classification_report

print(classification_report(pred,y_test))